In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import time

import numpy as np
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import svds

import torch
from input_data import load_data
from preprocessing import general_train_test_split_edges,biased_train_test_split_edges, bidirectional_train_test_split_edges
from layers import DirectedGCNConvEncoder, DirectedInnerProductDecoder
from layers import SingleLayerDirectedGCNConvEncoder
from models import DirectedGAE

from layers import InnerProductDecoder
from layers import DummyEncoder, DummyPairEncoder

from models import GAE

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset',
                    nargs= '?',
                    default='cornell',
                    type=str)
parser.add_argument('--task',
                    nargs= '?',
                    default='task_3',
                    type=str)
parser.add_argument('--model',
                    nargs= '?',
                    default='digae',
                    type=str)
parser.add_argument('--learning_rate',
                    nargs= '?',
                    default=0.01,
                    type=float)
parser.add_argument('--epochs',
                    nargs= '?',
                    default=200,
                    type=int)
parser.add_argument('--hidden',
                    nargs= '?',
                    default=64,
                    type=int)
parser.add_argument('--dimension',
                    nargs= '?',
                    default=32,
                    type=int)
parser.add_argument('--alpha',
                    nargs= '?',
                    default=1.0,
                    type=float)
parser.add_argument('--beta',
                    nargs= '?',
                    default=0.0,
                    type=float)
parser.add_argument('--nb_run',
                    nargs= '?',
                    default=10,
                    type=int)
parser.add_argument('--prop_val',
                    nargs= '?',
                    default=5.0,
                    type=float)
parser.add_argument('--prop_test',
                    nargs= '?',
                    default=10.0,
                    type=float)
parser.add_argument('--verbose',
                    nargs= '?',
                    default=True,
                    type=bool)

parser.add_argument('--self_loops',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--adaptive',
                    nargs= '?',
                    default=False,
                    type=bool)
parser.add_argument('--feature_vector_type',
                    nargs= '?',
                    const=None)
parser.add_argument('--feature_vector_size',
                    nargs= '?',
                    const=None,
                    type=int)
parser.add_argument('--directed',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--logfile',
                    nargs='?',
                    default='logs.json',
                    type=str)
parser.add_argument('--validate',
                    nargs='?',
                    default=False,
                    type=bool)

args,_ = parser.parse_known_args()

In [3]:
def train_single():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

def dummy_train_single():
    model.train()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    return float(loss)

def dummy_test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)
    

def train_pair():
    model.train()
    optimizer.zero_grad()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def dummy_train_pair():
    model.train()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    return float(loss)

def dummy_test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def identity_features(data, k=None):
    num_nodes = data.x.size(0)
    x = torch.eye(num_nodes)
    u_hat = x.clone()
    v_hat = x.clone()
    return u_hat, v_hat

In [4]:
dummy_run = False
 
if args.verbose:
    print("Loading data...")

device     = 'cpu'
directed   = args.directed

dataset_name    = args.dataset
out_channels    = args.dimension
hidden_channels = args.hidden

compute_features = identity_features

feature_vector_size = args.feature_vector_size

if args.model in ['gae','source_target']:
    train_func = train_single
    test_func  = test_single
elif args.model in ['dummy']:
    train_func = dummy_train_single
    test_func  = dummy_test_single
elif args.model in ['dummy_pair']:
    train_func = dummy_train_pair
    test_func  = dummy_test_pair
else:
    train_func = train_pair
    test_func  = test_pair

val_ratio  = args.prop_val / 100.
test_ratio = args.prop_test / 100.

loaded_data = load_data(dataset_name, directed=directed)

Loading data...


In [5]:
mean_roc  = []
mean_ap   = []
mean_time = []

for i in range(args.nb_run):
    if args.verbose:
        print("Masking test edges...")

    if args.task == 'task_1':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = general_train_test_split_edges(data,
                                                         val_ratio=val_ratio,
                                                         test_ratio=test_ratio,
                                                         directed=directed)
    elif args.task == 'task_2':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = biased_train_test_split_edges(data,
                                                        val_ratio=val_ratio,
                                                        test_ratio=test_ratio,
                                                        directed=directed)
    elif args.task == 'task_3':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = bidirectional_train_test_split_edges(data,
                                                               val_ratio=val_ratio,
                                                               test_ratio=test_ratio,
                                                               directed=directed)     
    else:
        raise ValueError('Undefined task!')

    data = data.to(device)
    train_pos_edge_index = data.train_pos_edge_index.to(device)
    if args.validate is True:
        test_pos_edge_index  = data.val_pos_edge_index.to(device)
        test_neg_edge_index  = data.val_neg_edge_index.to(device)
    else:
        test_pos_edge_index  = data.test_pos_edge_index.to(device)
        test_neg_edge_index  = data.test_neg_edge_index.to(device)

    in_channels          = data.x.shape[1]

    in_channels = data.x.size(0)
    u, v = compute_features(data, in_channels)
    data.u = u
    data.v = v
    data.x = torch.cat([data.u, data.v], dim=1)
    
    u = data.u.to(device)
    v = data.v.to(device)
    print(u.shape, v.shape)

    x = data.x.to(device)

    if args.model == 'digae':
        encoder = DirectedGCNConvEncoder(in_channels, hidden_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                         self_loops=args.self_loops,
                                         adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'digae_single_layer':
        encoder = SingleLayerDirectedGCNConvEncoder(in_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                                    self_loops=args.self_loops,
                                                    adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'dummy':
        encoder = DummyEncoder()
        decoder = InnerProductDecoder()
        model   = GAE(encoder, decoder)
        model   = model.to(device)
        dummy_run = True

    elif args.model == 'dummy_pair':
        encoder = DummyPairEncoder()
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)
        dummy_run =True
    else:
        raise ValueError('Undefined model!')

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)

    if args.verbose:
        print("Training...")

    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(args.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        loss = train_func()
        avg_cost = loss
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
        "time=", "{:.5f}".format(time.time() - t))
    mean_time.append(time.time() - t_start)

    if args.verbose:
        print("Testing model...")
    auc, ap = test_func(test_pos_edge_index, test_neg_edge_index)
    roc_score, ap_score = auc, ap
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# if adaptive...
print('=' * 60)
print(args.adaptive)
print(args.model)
print('=' * 60)
    
# Report final results
print("\nTest results for", args.model,
      "model on", args.dataset, "on", args.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Masking test edges...
torch.Size([183, 183]) torch.Size([183, 183])
Training...
Epoch: 0001 train_loss= 1.38720 time= 0.00999
Epoch: 0002 train_loss= 1.38554 time= 0.00700
Epoch: 0003 train_loss= 1.38438 time= 0.00500
Epoch: 0004 train_loss= 1.38162 time= 0.00365
Epoch: 0005 train_loss= 1.37693 time= 0.00400
Epoch: 0006 train_loss= 1.36917 time= 0.00500
Epoch: 0007 train_loss= 1.35590 time= 0.00382
Epoch: 0008 train_loss= 1.33795 time= 0.00516
Epoch: 0009 train_loss= 1.31692 time= 0.00500
Epoch: 0010 train_loss= 1.30141 time= 0.00403
Epoch: 0011 train_loss= 1.27846 time= 0.00267
Epoch: 0012 train_loss= 1.27233 time= 0.00441
Epoch: 0013 train_loss= 1.24721 time= 0.00205
Epoch: 0014 train_loss= 1.21796 time= 0.00598
Epoch: 0015 train_loss= 1.20753 time= 0.00105
Epoch: 0016 train_loss= 1.19929 time= 0.00106
Epoch: 0017 train_loss= 1.22052 time= 0.00450
Epoch: 0018 train_loss= 1.15175 time= 0.00205
Epoch: 0019 train_loss= 1.17801 time= 0.00106
Epoch: 0020 train_loss= 1.13483 time= 0.00487


Epoch: 0191 train_loss= 0.32938 time= 0.00599
Epoch: 0192 train_loss= 0.33055 time= 0.00793
Epoch: 0193 train_loss= 0.32872 time= 0.00712
Epoch: 0194 train_loss= 0.36554 time= 0.00510
Epoch: 0195 train_loss= 0.39319 time= 0.00505
Epoch: 0196 train_loss= 0.33026 time= 0.00475
Epoch: 0197 train_loss= 0.35464 time= 0.00390
Epoch: 0198 train_loss= 0.29015 time= 0.00599
Epoch: 0199 train_loss= 0.35631 time= 0.00600
Epoch: 0200 train_loss= 0.34360 time= 0.00161
Testing model...
Masking test edges...
torch.Size([183, 183]) torch.Size([183, 183])
Training...
Epoch: 0001 train_loss= 1.38652 time= 0.00601
Epoch: 0002 train_loss= 1.38558 time= 0.00694
Epoch: 0003 train_loss= 1.38352 time= 0.00539
Epoch: 0004 train_loss= 1.38005 time= 0.00562
Epoch: 0005 train_loss= 1.37533 time= 0.00700
Epoch: 0006 train_loss= 1.36884 time= 0.00702
Epoch: 0007 train_loss= 1.35456 time= 0.00592
Epoch: 0008 train_loss= 1.34635 time= 0.00605
Epoch: 0009 train_loss= 1.33073 time= 0.00548
Epoch: 0010 train_loss= 1.289

Epoch: 0171 train_loss= 0.35275 time= 0.00800
Epoch: 0172 train_loss= 0.40200 time= 0.00500
Epoch: 0173 train_loss= 0.31702 time= 0.00520
Epoch: 0174 train_loss= 0.33120 time= 0.00700
Epoch: 0175 train_loss= 0.35520 time= 0.00484
Epoch: 0176 train_loss= 0.35751 time= 0.00484
Epoch: 0177 train_loss= 0.37293 time= 0.00500
Epoch: 0178 train_loss= 0.34695 time= 0.00600
Epoch: 0179 train_loss= 0.36032 time= 0.00700
Epoch: 0180 train_loss= 0.37829 time= 0.00600
Epoch: 0181 train_loss= 0.30050 time= 0.00500
Epoch: 0182 train_loss= 0.31783 time= 0.00600
Epoch: 0183 train_loss= 0.31481 time= 0.00538
Epoch: 0184 train_loss= 0.34321 time= 0.00473
Epoch: 0185 train_loss= 0.38074 time= 0.00500
Epoch: 0186 train_loss= 0.33447 time= 0.00600
Epoch: 0187 train_loss= 0.32599 time= 0.00601
Epoch: 0188 train_loss= 0.37035 time= 0.00499
Epoch: 0189 train_loss= 0.34943 time= 0.00401
Epoch: 0190 train_loss= 0.29250 time= 0.00500
Epoch: 0191 train_loss= 0.36434 time= 0.00567
Epoch: 0192 train_loss= 0.34590 ti

Epoch: 0156 train_loss= 0.41359 time= 0.00500
Epoch: 0157 train_loss= 0.32051 time= 0.00521
Epoch: 0158 train_loss= 0.34762 time= 0.00700
Epoch: 0159 train_loss= 0.31382 time= 0.00706
Epoch: 0160 train_loss= 0.29010 time= 0.00594
Epoch: 0161 train_loss= 0.32120 time= 0.00363
Epoch: 0162 train_loss= 0.37111 time= 0.00404
Epoch: 0163 train_loss= 0.34380 time= 0.00455
Epoch: 0164 train_loss= 0.36440 time= 0.00401
Epoch: 0165 train_loss= 0.38565 time= 0.00400
Epoch: 0166 train_loss= 0.31371 time= 0.00464
Epoch: 0167 train_loss= 0.35164 time= 0.00422
Epoch: 0168 train_loss= 0.32684 time= 0.00407
Epoch: 0169 train_loss= 0.35334 time= 0.00463
Epoch: 0170 train_loss= 0.34078 time= 0.00500
Epoch: 0171 train_loss= 0.40502 time= 0.00400
Epoch: 0172 train_loss= 0.37915 time= 0.00416
Epoch: 0173 train_loss= 0.36820 time= 0.00500
Epoch: 0174 train_loss= 0.31566 time= 0.00500
Epoch: 0175 train_loss= 0.35606 time= 0.00335
Epoch: 0176 train_loss= 0.32372 time= 0.00601
Epoch: 0177 train_loss= 0.33756 ti

Epoch: 0145 train_loss= 0.42335 time= 0.00800
Epoch: 0146 train_loss= 0.34618 time= 0.00602
Epoch: 0147 train_loss= 0.41647 time= 0.00714
Epoch: 0148 train_loss= 0.33553 time= 0.00584
Epoch: 0149 train_loss= 0.39457 time= 0.00334
Epoch: 0150 train_loss= 0.35291 time= 0.00494
Epoch: 0151 train_loss= 0.39418 time= 0.00600
Epoch: 0152 train_loss= 0.42590 time= 0.00600
Epoch: 0153 train_loss= 0.42060 time= 0.00666
Epoch: 0154 train_loss= 0.38118 time= 0.00622
Epoch: 0155 train_loss= 0.38067 time= 0.00560
Epoch: 0156 train_loss= 0.39430 time= 0.00501
Epoch: 0157 train_loss= 0.32552 time= 0.00500
Epoch: 0158 train_loss= 0.34229 time= 0.00600
Epoch: 0159 train_loss= 0.39654 time= 0.00607
Epoch: 0160 train_loss= 0.32230 time= 0.00600
Epoch: 0161 train_loss= 0.37011 time= 0.00703
Epoch: 0162 train_loss= 0.30527 time= 0.00507
Epoch: 0163 train_loss= 0.36855 time= 0.00613
Epoch: 0164 train_loss= 0.33159 time= 0.00679
Epoch: 0165 train_loss= 0.35247 time= 0.00600
Epoch: 0166 train_loss= 0.32958 ti

...

Epoch: 0125 train_loss= 0.43275 time= 0.00706
Epoch: 0126 train_loss= 0.43831 time= 0.00600
Epoch: 0127 train_loss= 0.44107 time= 0.00594
Epoch: 0128 train_loss= 0.42020 time= 0.00707
Epoch: 0129 train_loss= 0.39129 time= 0.00621
Epoch: 0130 train_loss= 0.38720 time= 0.00776
Epoch: 0131 train_loss= 0.38696 time= 0.00496
Epoch: 0132 train_loss= 0.42279 time= 0.00799
Epoch: 0133 train_loss= 0.42135 time= 0.00701
Epoch: 0134 train_loss= 0.39806 time= 0.00542
Epoch: 0135 train_loss= 0.41776 time= 0.00700
Epoch: 0136 train_loss= 0.43126 time= 0.00598
Epoch: 0137 train_loss= 0.38865 time= 0.00802
Epoch: 0138 train_loss= 0.39025 time= 0.00601
Epoch: 0139 train_loss= 0.44483 time= 0.00700
Epoch: 0140 train_loss= 0.40875 time= 0.00505
Epoch: 0141 train_loss= 0.35992 time= 0.00347
Epoch: 0142 train_loss= 0.41638 time= 0.00404
Epoch: 0143 train_loss= 0.39243 time= 0.00706
Epoch: 0144 train_loss= 0.40601 time= 0.00681
Epoch: 0145 train_loss= 0.36158 time= 0.00706
Epoch: 0146 train_loss= 0.32438 ti

Epoch: 0105 train_loss= 0.43036 time= 0.00806
Epoch: 0106 train_loss= 0.39656 time= 0.00633
Epoch: 0107 train_loss= 0.39572 time= 0.00598
Epoch: 0108 train_loss= 0.49787 time= 0.00503
Epoch: 0109 train_loss= 0.45025 time= 0.00599
Epoch: 0110 train_loss= 0.50117 time= 0.00424
Epoch: 0111 train_loss= 0.38932 time= 0.00398
Epoch: 0112 train_loss= 0.40780 time= 0.00492
Epoch: 0113 train_loss= 0.43845 time= 0.00492
Epoch: 0114 train_loss= 0.37233 time= 0.00382
Epoch: 0115 train_loss= 0.44917 time= 0.00529
Epoch: 0116 train_loss= 0.45845 time= 0.00601
Epoch: 0117 train_loss= 0.41419 time= 0.00656
Epoch: 0118 train_loss= 0.45538 time= 0.00607
Epoch: 0119 train_loss= 0.49877 time= 0.00600
Epoch: 0120 train_loss= 0.44265 time= 0.00697
Epoch: 0121 train_loss= 0.44381 time= 0.00496
Epoch: 0122 train_loss= 0.40901 time= 0.00498
Epoch: 0123 train_loss= 0.39348 time= 0.00741
Epoch: 0124 train_loss= 0.42526 time= 0.00500
Epoch: 0125 train_loss= 0.40646 time= 0.00594
Epoch: 0126 train_loss= 0.31782 ti

Epoch: 0090 train_loss= 0.48642 time= 0.00696
Epoch: 0091 train_loss= 0.48971 time= 0.00500
Epoch: 0092 train_loss= 0.48998 time= 0.00494
Epoch: 0093 train_loss= 0.45500 time= 0.00701
Epoch: 0094 train_loss= 0.46571 time= 0.00594
Epoch: 0095 train_loss= 0.45417 time= 0.00605
Epoch: 0096 train_loss= 0.49581 time= 0.00602
Epoch: 0097 train_loss= 0.45210 time= 0.00696
Epoch: 0098 train_loss= 0.44909 time= 0.00542
Epoch: 0099 train_loss= 0.41155 time= 0.00598
Epoch: 0100 train_loss= 0.43609 time= 0.00703
Epoch: 0101 train_loss= 0.48039 time= 0.00526
Epoch: 0102 train_loss= 0.46019 time= 0.00611
Epoch: 0103 train_loss= 0.44619 time= 0.00587
Epoch: 0104 train_loss= 0.45707 time= 0.00602
Epoch: 0105 train_loss= 0.42121 time= 0.00594
Epoch: 0106 train_loss= 0.40697 time= 0.00606
Epoch: 0107 train_loss= 0.43476 time= 0.00601
Epoch: 0108 train_loss= 0.38083 time= 0.00696
Epoch: 0109 train_loss= 0.40414 time= 0.00605
Epoch: 0110 train_loss= 0.38664 time= 0.00597
Epoch: 0111 train_loss= 0.44133 ti

Epoch: 0077 train_loss= 0.65675 time= 0.01164
Epoch: 0078 train_loss= 0.67867 time= 0.00666
Epoch: 0079 train_loss= 0.58844 time= 0.00500
Epoch: 0080 train_loss= 0.59031 time= 0.00600
Epoch: 0081 train_loss= 0.56940 time= 0.00600
Epoch: 0082 train_loss= 0.58724 time= 0.00600
Epoch: 0083 train_loss= 0.57215 time= 0.00600
Epoch: 0084 train_loss= 0.53021 time= 0.00600
Epoch: 0085 train_loss= 0.50959 time= 0.00700
Epoch: 0086 train_loss= 0.53402 time= 0.00301
Epoch: 0087 train_loss= 0.57062 time= 0.00106
Epoch: 0088 train_loss= 0.53917 time= 0.00519
Epoch: 0089 train_loss= 0.48171 time= 0.00595
Epoch: 0090 train_loss= 0.48084 time= 0.00601
Epoch: 0091 train_loss= 0.53940 time= 0.00701
Epoch: 0092 train_loss= 0.54460 time= 0.00597
Epoch: 0093 train_loss= 0.44780 time= 0.00602
Epoch: 0094 train_loss= 0.44985 time= 0.00700
Epoch: 0095 train_loss= 0.45504 time= 0.00601
Epoch: 0096 train_loss= 0.54016 time= 0.00589
Epoch: 0097 train_loss= 0.53280 time= 0.00710
Epoch: 0098 train_loss= 0.47777 ti

Epoch: 0062 train_loss= 0.73595 time= 0.00700
Epoch: 0063 train_loss= 0.72124 time= 0.00474
Epoch: 0064 train_loss= 0.70361 time= 0.00402
Epoch: 0065 train_loss= 0.63767 time= 0.00594
Epoch: 0066 train_loss= 0.66454 time= 0.00494
Epoch: 0067 train_loss= 0.67480 time= 0.00399
Epoch: 0068 train_loss= 0.64836 time= 0.00492
Epoch: 0069 train_loss= 0.65006 time= 0.00607
Epoch: 0070 train_loss= 0.62468 time= 0.00600
Epoch: 0071 train_loss= 0.65981 time= 0.00702
Epoch: 0072 train_loss= 0.64607 time= 0.00699
Epoch: 0073 train_loss= 0.51440 time= 0.00595
Epoch: 0074 train_loss= 0.61498 time= 0.00703
Epoch: 0075 train_loss= 0.68277 time= 0.00601
Epoch: 0076 train_loss= 0.63725 time= 0.00592
Epoch: 0077 train_loss= 0.52192 time= 0.00506
Epoch: 0078 train_loss= 0.57298 time= 0.00501
Epoch: 0079 train_loss= 0.59596 time= 0.00700
Epoch: 0080 train_loss= 0.51078 time= 0.00602
Epoch: 0081 train_loss= 0.50877 time= 0.00559
Epoch: 0082 train_loss= 0.60711 time= 0.00598
Epoch: 0083 train_loss= 0.60965 ti